In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install pandas
!pip install numpy==1.23.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 12.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661657 sha256=f942c4972dcbb760f61f3c470e31da7373417c790cebed31f85902d0c3fa2743
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 81.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are insta

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-59is3n1t/kobert-tokenizer_a2e18a8413894b54b56e286d54a72f8d
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-59is3n1t/kobert-tokenizer_a2e18a8413894b54b56e286d54a72f8d
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=037f7193ba5a7116e7be2ab7d24d2d50ad7cd4c52bd8548a9a25e8e59f517847
  Stored in directory: /tmp/pip-ephem-wheel-cache-6bznsjdk/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [ ]:
#  Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 설정
device = torch.device("cuda:0")

In [ ]:
from google.colab import drive
import json
import pandas as pd

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = '/content/drive/MyDrive/processed_data_immoral/Training/processed_training_data.json'

In [ ]:
with open(data_path, 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)

print(df.head())

                                                text  is_immoral  intensity  \
0  치어리더가 숏컷했다고 페미라는 소리하는 한남 누구야, 애당초 치어가 페미 좋아할 수...        True        2.0   
1                                    한녀는 무조건 피싸게 패미임        True        1.8   
2       패미가 격투기 라운드걸이랑 치어리더는 밥줄 끊으려고 했는데 좋아할수가 없지않나?        True        1.8   
3            그럴리가 없잖음, 한녀가 얼마나 전통적으로 피빠는데, 기생이 그 증거임        True        1.8   
4  너 같은 2030 무뇌키보충이 참 걱정임, 페미덕에 레이싱걸 없어졌는데 참으로 치어...        True        2.0   

       types  
0  [CENSURE]  
1  [CENSURE]  
2  [CENSURE]  
3  [CENSURE]  
4  [CENSURE]  


In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        text = row['text']
        is_immoral = row['is_immoral']
        intensity = row['intensity']

        # Tokenization
        encoding = self.tokenizer(
            text,
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'is_immoral': torch.tensor(is_immoral, dtype=torch.float),
            'intensity': torch.tensor(intensity, dtype=torch.float),
        }

In [ ]:
class KoBERTClassifier(nn.Module):
    def __init__(self):
        super(KoBERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('skt/kobert-base-v1')
        self.dropout = nn.Dropout(p=0.3)
        self.fc_is_immoral = nn.Linear(self.bert.config.hidden_size, 1)
        self.fc_intensity = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)

        # 출력 정의
        is_immoral = self.fc_is_immoral(x)
        intensity = self.fc_intensity(x)

        # 강도 값을 1.0~3.0으로 제한
        intensity = torch.clamp(intensity, min=1.0, max=3.0)

        return is_immoral, intensity

In [ ]:
# Hyperparameters
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
dataset = CustomDataset(df, tokenizer, max_len=128)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

model = KoBERTClassifier().to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn_is_immoral = nn.BCEWithLogitsLoss()
loss_fn_intensity = nn.MSELoss()
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=100,
    num_training_steps=len(dataloader) * 5
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
def train_epoch(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        is_immoral = batch['is_immoral'].to(device)
        intensity = batch['intensity'].to(device)

        optimizer.zero_grad()

        # Forward pass
        preds_is_immoral, preds_intensity = model(input_ids, attention_mask)

        # 손실 계산
        loss_is_immoral = loss_fn_is_immoral(preds_is_immoral.squeeze(-1), is_immoral)
        loss_intensity = loss_fn_intensity(preds_intensity.squeeze(-1), intensity)

        # 총 손실 계산
        loss = loss_is_immoral + loss_intensity

        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

# Training Loop
EPOCHS = 5
for epoch in range(EPOCHS):
    train_loss = train_epoch(model, dataloader, optimizer, scheduler, device)
    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {train_loss:.4f}")

100%|██████████| 4019/4019 [22:26<00:00,  2.98it/s]


Epoch 1/5, Loss: 1.0926


100%|██████████| 4019/4019 [22:24<00:00,  2.99it/s]


Epoch 2/5, Loss: 0.9858


100%|██████████| 4019/4019 [22:24<00:00,  2.99it/s]


Epoch 3/5, Loss: 0.8810


100%|██████████| 4019/4019 [22:24<00:00,  2.99it/s]


Epoch 4/5, Loss: 0.7869


100%|██████████| 4019/4019 [22:24<00:00,  2.99it/s]

Epoch 5/5, Loss: 0.7420


In [ ]:
import os

save_path = '/content/drive/MyDrive/koBERT_model/kobert_model.pth'
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")

Model saved to /content/drive/MyDrive/koBERT_model/kobert_model.pth
